<a href="https://colab.research.google.com/github/javmohir/Machine_Learning/blob/main/ML_pridect_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn # scikit-learn kutubxonasi

Machine Learning
Nihoyat ma'lumotlarimiz ML uchun tayyor.

Boshlanishiga kerakli modullar va ma'lumotlarni qayta o'qib olamiz.

Ma'lumotlarni o'qiymiz

In [ ]:
# Onlayn dataset joylashgan manzilini ko'rsatamiaz
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)

Ma'lumotlarni train va testga ajratamiz.

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_set.drop("median_house_value", axis=1)
y = train_set["median_house_value"].copy()

X_num = X_train.drop("ocean_proximity", axis=1)


Pipeline quramiz

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
# bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        rooms_per_household =  X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix]/X[:, households_ix]
        if self.add_bedrooms_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
             bedrooms_per_room = X[:, bedrooms_ix]/X[:, rooms_ix]
             return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

Sonli ustunlar uchun

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder(add_bedrooms_per_room=True)),
    ('std_scaler', StandardScaler())
])

Matnli ustunlar uchun

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [ ]:
from sklearn.compose import  ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

Mana yakuniy, to'liq konveyer tayyor bo'ldi (full_pipeline).

Konveyerni ishga tushirish uchun .fit_transform() metodini chaqrisih kifoya.

In [ ]:
X_perpared = full_pipeline.fit_transform(X_train)

In [ ]:
X_perpared[0:5, :]

Ma'lumotlar ML uchun tayyor.

Machine Learning
* Bizning maqsadimiz bashorat qilish, buning uchun bir nechta ML algoritmlar mavjud.

Biz kelgusi darslarda ularning har biri bilan yaqinda tanishamiz, hozir esa scikit-learn tarkibidagi ba'zi tayyor algoritmlardan foydalanamiz.

Linear Regression - Chiziqli regressiya
sklearn tarkibidagi LinearRegression klassidan yangi model yaratamiz.

In [ ]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()

LinearRegression bu estimator. Estimatorlar ma'lumotlarni qabul qilib oladi va .fit() metodi yordamida ulardan basorat qilishni o'rganadi (machine learning)

In [ ]:
LR_model.fit(X_perpared, y)

TAMOM! Machine Learning tugadi! Ha, adashmadingiz, bor yo'g'i 3 qator kod bilan biz hozirgina kompyuterga uylarni narxini bashorat qilishni o'rgatdik.

Modelni qanday qilib tekshirib ko'rishimiz mumkin? Keling housing datasetdan biror qatorni modelga beramiz va chiqqan natijani bizdagi bor natija (label) bilan solishtiramiz.

In [ ]:
test_data = X_train.sample(5)
test_data

In [ ]:
# yuqoridagi qatorlarga mos keluvchi narxlarni ajratib olamiz (biz aynan shu qiymatlarni bashorat qilishimiz kerak)
test_label = y.loc[test_data.index]
test_label

test_data ni pipelinedan o'tkazib, bizga kerak ko'rinishga keltirib olamiz.

Ahamiyat bering bu safgar biz .transform() metodini chaqiramiz, sababi .fit() metodini avval chaqirgan edik.

In [ ]:
test_data_prepared = full_pipeline.transform(test_data)
test_data_prepared

Bashorat (predict) qilamiz:

In [ ]:
predicted_data = LR_model.predict(test_data_prepared)
predicted_data

Yuoqirda ko'rib turganingiz bashorat qilingan qiymatlar. Xo'sh, ular real qiymatlardan qanday farq qiladi, solishtiramiz:

In [ ]:
pd.DataFrame({'prognoz':predicted_data, 'asl bahosi':test_label})

5-QADAM. Modelni baholaymiz
Ko'rib turganingizdek, modelimiz qayerdadur kamroq, qayeradur ko'proq xato bilan bashorat qilgan.
Lekin model aniqligini baxolash uchun 5 qator yetarli emas. Keling, avvalroq ajratib olgan test set yordamida sinab ko'ramiz:

In [ ]:
test_set

Avval predictor ustunlarni ajratib olaylik (median_house_value) dan boshqa barcha ustunlar):

In [ ]:
X_test = test_set.drop("median_house_value", axis=1)
X_test

Label (median_house_value) ustunini ajratib olamiz.

In [ ]:
y_test = test_set['median_house_value'].copy()
y_test

test_set ni ham pipelinedan o'tkazamiz:

In [ ]:
X_test_prepared = full_pipeline.transform(X_test)

In [ ]:
y_pridect = LR_model.predict(X_test_prepared)

Bashorat va real datani solishtirish uchun avvalgi bo'limda ko'rgan Root mean square error (RMSE) dan foydalanamiz:

In [ ]:
# O'rtacha absolyut xatolik
from sklearn.metrics import mean_absolute_error
mea = mean_absolute_error(y_test, y_pridect)
print("MEA=", mea)

In [ ]:
# O'rtacha kvadrad xatolikni hisoblash
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pridect)
print("MSE=", np.sqrt(mse))

Demak, RMSE=72701$ chiqdi. Yomon emas, lekin yaxshi ham emas. Ya'ni modelimiz uylarni baholashda o'rtacha 72000$ ga adashayapti.

Model aniqligini oshirish uchun yagona, universal yechim yo'q. Qilib ko'rishingiz mumkin bo'lgan ishlar:

Yaxhsiroq paramterlar topish
Yaxhsiroq model (algoritm) tanlash
Ko'proq ma'lumot yig'ish va hokazo.
Biz hozir boshqa model bilan sinab ko'ramiz.

DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
Tree_model = DecisionTreeClassifier()
Tree_model.fit(X_perpared, y)

Modelni tekshiramiz:

In [ ]:
y_pridect = Tree_model.predict(X_test_prepared)

In [ ]:
lin_mse = mean_squared_error(y_test, y_pridect)
print("mes=", np.sqrt(lin_mse))

RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(X_perpared, y)

Modelni tekshiramiz:

In [ ]:
y_predicted = RF_model.predict(X_test_prepared)
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

In [ ]:
y_pridect = RF_model.predict(X_test_prepared)
lin_a = mean_squared_error(y_test, y_pridect)
print("MSE=", np.sqrt(lin_a))

Avvalgidan yaxhsiroq.

Cross-Validation usuli bilan baholash
Yuqorida biz modelni baholash uchun ma'lumotlarni test va train setlarga ajratdik.
Bu usulning kamchiligi biz test va train uchun doim bir xil ma'lumotlardan foydalanayapmiz.

Cross-validation yordamida biz ma'lumotlarni bir necha qismga ajratib, modelni turli qismlar yordamida bir nechta bor train va test qilishimiz mumkin.

Misol uchun, quyidagi rasmda ma'lumotlarni 5 ga ajratib train va test qilish ko'rsatilgan.

Сross validation uchun ma'lumotlarni train va testga bo'lish shart emas, buni sklearn o'zi qiladi.

In [ ]:
X = df.drop("median_house_value", axis=1)
y = df['median_house_value'].copy()
X_perpared = full_pipeline.transform(X)

Validation natijalarini ko'rsatish uchun sodda funksiya yasab olamiz

In [ ]:
def display_scores(scores):
    print("Score:", scores)
    print("Mean:", scores.mean())
    print("Std.dev:", scores.std())


Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score


LogisticRegression

In [ ]:
scores = cross_val_score(LR_model, X_perpared,  y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)

In [ ]:
display_scores(LR_rmse_scores)

In [ ]:
scores = cross_val_score(RF_model, X_perpared, y, scoring="neg_mean_squared_error", cv=5)
RF_rmse_scores = np.sqrt(-scores)
display_scores(RF_rmse_scores)

Modelni saqlash
Yaratgan modelimizdan kelajakda foydalanish uchun saqlab qo'yishimiz lozim. Umuman olganda nafaqat model, balki boshqa kerak bo'ladigan o'zgaruvchilarni ham saqlab qo'yish maqsadga muvvofiq bo'ladi. Masalan pipeline.

Buning uchun Pythondagi pickle yoki joblib modullaridan foydalanamiz.

* pickle yordamida saqlash

In [ ]:
import pickle
filename = 'RF_model.pkl' # faylga istalgan nom beramiz
with open(filename, 'wb') as file:
    pickle.dump(RF_model, file)

Modelni qayta o'qiymiz:

In [ ]:
with open(filename, 'rb') as file:
    model=pickle.load(file)

Modelni sinab ko'ramiz

In [ ]:
scores = cross_val_score(model, X_perpared, y, scoring="neg_mean_squared_error", cv=5)
RF_rmse_scores = np.sqrt(-scores)
display_scores(RF_rmse_scores)

joblib yordamida saqlash
joblib katta NumPy martrisalarni siqib saqlash uchun afzal.

joblib o'rnatilmagan bo'lsa pip install joblib yordamida o'rnatib oling.

In [ ]:
import joblib

filename = 'LR_model.jbl' # faylga istalgan nom beramiz
joblib.dump(LR_model, filename)

Modelni o'qiymiz

In [ ]:
madel = joblib.load(filename)

In [ ]:
scores = cross_val_score(madel, X_perpared, y, scoring="neg_mean_squared_error", cv=5)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

In [ ]:
filenom = 'pipeline.jbl'
joblib.dump(full_pipeline,  filenom)

In [ ]:
pipl = joblib.load(filenom)

In [ ]:
X_prepared = full_pipeline.fit_transform(X_train)

In [ ]:
X_tex = pipl.fit_transform(X_train)
X_tex